In [6]:
import pandas as pd
import pdfplumber
import os

# ==============================================================================
# PARTE 1: PROCESSAR O RUNDOWN (PDF -> DATAFRAME)
# ==============================================================================

def processar_rundown_pdf(caminho_pdf):
    """
    Função estável para extrair os dados do rundown (espelho) de um PDF,
    deduzindo os blocos a partir das linhas de 'Break'.
    """
    if not os.path.exists(caminho_pdf):
        print(f"ERRO: O arquivo de rundown PDF não foi encontrado em '{caminho_pdf}'")
        return None

    dados_finais = []
    bloco_numero = 1

    with pdfplumber.open(caminho_pdf) as pdf:
        for pagina in pdf.pages:
            tabelas = pagina.extract_tables()
            for tabela in tabelas:
                for linha in tabela:
                    if linha and linha[0] and isinstance(linha[0], str) and 'Break' in linha[0]:
                        bloco_numero += 1
                        continue
                    
                    if len(linha) >= 10 and linha[-1] and isinstance(linha[-1], str) and linha[-1].isdigit():
                        dados_finais.append({
                            'Bloco': f"Bloco {bloco_numero:02d}",
                            'Tipo': linha[1],
                            'Retranca': linha[2],
                            'Início (Lauda)': linha[6],
                            'Fim (Lauda)': linha[7],
                            'Ordem': int(linha[9])
                        })

    if not dados_finais:
        print("Atenção: Nenhum dado de lauda válido foi extraído do PDF.")
        return None

    df = pd.DataFrame(dados_finais)
    df = df.sort_values(by='Ordem').reset_index(drop=True)
    
    print("PDF do rundown processado com sucesso!")
    return df

# ==============================================================================
# PARTE 2: FAZER O MERGE COM DADOS DE AUDIÊNCIA (VERSÃO FINAL)
# ==============================================================================

def fazer_merge_audiencia_espelho(df_espelho_orig, caminho_audiencia):
    """
    Recebe o DataFrame do espelho, o caminho do arquivo de audiência,
    e realiza a expansão e o merge em nível de segundo.
    """
    # --- Carregar o dataframe de audiência ---
    try:
        nome_arquivo, extensao = os.path.splitext(caminho_audiencia)
        if extensao.lower() == '.csv':
            df_minamin_orig = pd.read_csv(caminho_audiencia, sep=';', on_bad_lines='skip')
        elif extensao.lower() in ['.xlsx', '.xls']:
            df_minamin_orig = pd.read_excel(caminho_audiencia)
        else:
            print(f"ERRO CRÍTICO: Formato de arquivo não suportado '{extensao}'.")
            return None
        
        print(f"Arquivo de audiência '{caminho_audiencia}' carregado com sucesso!")

    except Exception as e:
        print(f"ERRO CRÍTICO ao ler o arquivo de audiência: {e}")
        return None

    # --- 1. Limpeza e Expansão do df_minamin ---
    print("Processando dados de audiência...")
    df_minamin = df_minamin_orig.copy()
    
    # AQUI ESTÁ A CORREÇÃO FINAL DO FORMATO DA HORA!
    df_minamin['HORA_base'] = pd.to_datetime(df_minamin['HORA'], format='%H:%M', errors='coerce').dt.time
    
    df_minamin.dropna(subset=['HORA_base'], inplace=True)

    if df_minamin.empty:
        print("[ALERTA CRÍTICO] O dataframe de audiência ficou vazio após a limpeza. Verifique o arquivo de origem e o formato da hora.")
        return None
    
    # Expansão para segundos
    df_minamin_segundos = df_minamin.copy()
    df_minamin_segundos['datetime_segundo_audiencia'] = df_minamin_segundos['HORA_base'].apply(
        lambda t: pd.date_range(start=f"1900-01-01 {t}", periods=60, freq='S')
    )
    df_minamin_segundos = df_minamin_segundos.explode('datetime_segundo_audiencia')

    # --- 2. Expansão do df_espelho ---
    print("Processando dados do espelho (rundown)...")
    df_espelho = df_espelho_orig.copy()
    df_espelho.rename(columns={'Início (Lauda)': 'inicio_lauda', 'Fim (Lauda)': 'fim_lauda'}, inplace=True)
    df_espelho['inicio_lauda_dt'] = pd.to_datetime('1900-01-01 ' + df_espelho['inicio_lauda'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
    df_espelho['fim_lauda_dt'] = pd.to_datetime('1900-01-01 ' + df_espelho['fim_lauda'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
    df_espelho.dropna(subset=['inicio_lauda_dt', 'fim_lauda_dt'], inplace=True)
    df_espelho = df_espelho[df_espelho['fim_lauda_dt'] >= df_espelho['inicio_lauda_dt']].copy()
    
    if not df_espelho.empty:
        df_espelho['datetime_segundo_espelho'] = df_espelho.apply(
            lambda row: pd.date_range(start=row['inicio_lauda_dt'], end=row['fim_lauda_dt'], freq='S'), axis=1
        )
        df_espelho_segundos = df_espelho.explode('datetime_segundo_espelho')
        df_espelho_segundos.drop_duplicates(subset=['datetime_segundo_espelho'], keep='first', inplace=True)
    else:
        print("[ALERTA] O dataframe do espelho ficou vazio após a limpeza.")
        return None

    # --- 3. Merge Final ---
    print("Realizando o merge dos dados...")
    df_merged_segundos = pd.merge(
        df_minamin_segundos,
        df_espelho_segundos,
        left_on='datetime_segundo_audiencia',
        right_on='datetime_segundo_espelho',
        how='left'
    )

    # --- 4. Limpeza e Salvamento ---
    colunas_para_remover = ['HORA_base', 'datetime_segundo_espelho', 'inicio_lauda_dt', 'fim_lauda_dt']
    df_merged_segundos.drop(columns=colunas_para_remover, inplace=True, errors='ignore')
    
    output_path = "audiencia_espelho_merged_segundos.csv"
    df_merged_segundos.to_csv(output_path, index=False, encoding='utf-8-sig')
    
    print(f"\nPROCESSO FINALIZADO! 🏆\nArquivo mergeado salvo em: {output_path}")
        
    return df_merged_segundos

# ==============================================================================
# PONTO DE PARTIDA PRINCIPAL DO SCRIPT
# ==============================================================================

if __name__ == "__main__":
    # --- Defina os nomes dos seus arquivos de entrada aqui ---
    ARQUIVO_RUNDOWN_PDF = "rundown-display.pdf"
    ARQUIVO_AUDIENCIA = "Realtime_clean.csv"

    print("Iniciando pipeline de processamento e merge...")
    
    # Passo 1: Processar o PDF para obter o espelho
    df_espelho_gerado = processar_rundown_pdf(ARQUIVO_RUNDOWN_PDF)
    
    # Passo 2: Se o espelho foi gerado, prosseguir com o merge
    if df_espelho_gerado is not None:
        df_final = fazer_merge_audiencia_espelho(df_espelho_gerado, ARQUIVO_AUDIENCIA)
        if df_final is not None and not df_final.empty:
            print("\nVisualização das primeiras linhas do dataframe final:")
            print(df_final.head())
    else:
        print("\nO pipeline foi interrompido porque o espelho não pôde ser gerado do PDF.")

Iniciando pipeline de processamento e merge...
PDF do rundown processado com sucesso!
Arquivo de audiência 'Realtime_clean.csv' carregado com sucesso!
Processando dados de audiência...
Processando dados do espelho (rundown)...
Realizando o merge dos dados...

PROCESSO FINALIZADO! 🏆
Arquivo mergeado salvo em: audiencia_espelho_merged_segundos.csv

Visualização das primeiras linhas do dataframe final:
    HORA GLOBO RECORD   SBT Conteúdo de TV/Vídeo sem referência  \
0  08:40  7,72   3,83  1,20                                8,23   
1  08:40  7,72   3,83  1,20                                8,23   
2  08:40  7,72   3,83  1,20                                8,23   
3  08:40  7,72   3,83  1,20                                8,23   
4  08:40  7,72   3,83  1,20                                8,23   

  Total Ligados Especial datetime_segundo_audiencia Bloco Tipo Retranca  \
0                  26,52        1900-01-01 08:40:00   NaN  NaN      NaN   
1                  26,52        1900-01-01 0

C:\Temp\ipykernel_18192\3716208346.py:92: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  lambda t: pd.date_range(start=f"1900-01-01 {t}", periods=60, freq='S')
C:\Temp\ipykernel_18192\3716208346.py:107: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  lambda row: pd.date_range(start=row['inicio_lauda_dt'], end=row['fim_lauda_dt'], freq='S'), axis=1
